In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/transformed/processed_odds.csv', index_col=0)

In [3]:
def get_class_vs_spread(x):
    if x[0] - x[1] > x[2] - x[3]:
        return 'home'
    elif x[0] - x[1] < x[2] - x[3]:
        return 'visitor'
    else:
        return 'push'

def get_class_vs_total(x):
    if x[0] + x[1] > x[2] + x[3]:
        return 'over'
    elif x[0] + x[1] < x[2] + x[3]:
        return 'under'
    else:
        return 'push'

In [4]:
df['open_spread_win'] = df[['home_score', 'visitor_score', 'home_score_open', 'visitor_score_open']].apply(lambda x: get_class_vs_spread(x), axis=1)
df['open_total_win'] = df[['home_score', 'visitor_score', 'home_score_open', 'visitor_score_open']].apply(lambda x: get_class_vs_total(x), axis=1)
df['close_spread_win'] = df[['home_score', 'visitor_score', 'home_score_close', 'visitor_score_open']].apply(lambda x: get_class_vs_spread(x), axis=1)
df['close_total_win'] = df[['home_score', 'visitor_score', 'home_score_close', 'visitor_score_close']].apply(lambda x: get_class_vs_total(x), axis=1)

In [5]:
dates = [d.split('_')[0] for d in df.index]
dates_final = []
for d in dates:
    y = d[:2]
    m = d[2:-2]
    d = d[-2:]
    if len(m) < 2:
        y = '20' + str(int(y)+1)
        m = '0' + m
    else:
        y = '20' + y
    dates_final += [y + m + d]

In [6]:
df['date'] = pd.to_datetime(dates_final)

In [7]:
teams = df['home_team'].unique()

In [8]:
home_prev_home = {}
home_prev_away = {}
visitor_prev_home = {}
visitor_prev_away = {}
for team in teams:
    print(team)
    df_curr = df[(df['home_team']==team) | (df['visitor_team']==team)]
    for i, game in enumerate(df_curr.index):
        home = False
        if df_curr.loc[game, 'home_team'] == team:
            home = True

        prev_home = False
        prev_away = False
        j = i
        while (j >= 0) & ((not prev_home) | (not prev_away)):
            if j == 0:
                if not prev_home:
                    prev_home = pd.Timestamp(year=2013, month=10, day=31)
                if not prev_away:
                    prev_away = pd.Timestamp(year=2013, month=10, day=31)

            idx = df_curr.index[j-1]
            if (not prev_home) & (df_curr.loc[idx, 'home_team'] == team):
                prev_home = df_curr.loc[idx, 'date']
            elif (not prev_away) & (df_curr.loc[idx, 'visitor_team'] == team):
                prev_away = df_curr.loc[idx, 'date']
            
            j -= 1
        
        if home:
            home_prev_home[game] = prev_home
            home_prev_away[game] = prev_away
        else:
            visitor_prev_home[game] = prev_home
            visitor_prev_away[game] = prev_away

Orlando
Charlotte
Washington
Atlanta
Boston
Houston
Memphis
Minnesota
Brooklyn
Denver
Phoenix
Sacramento
LALakers
Indiana
Philadelphia
NewOrleans
Milwaukee
Dallas
Utah
Portland
GoldenState
Detroit
Miami
OklahomaCity
NewYork
Cleveland
LAClippers
Toronto
SanAntonio
Chicago


In [9]:
df['home_prev_home'] = pd.Series(home_prev_home)
df['home_prev_away'] = pd.Series(home_prev_away)
df['visitor_prev_home'] = pd.Series(visitor_prev_home)
df['visitor_prev_away'] = pd.Series(visitor_prev_away)

In [35]:
df['home_days_since_home'] = ((df['date'] - df['home_prev_home']).dt.days).clip(lower=1, upper=15)
df['home_days_since_away'] = ((df['date'] - df['home_prev_away']).dt.days).clip(lower=1, upper=15)
df['visitor_days_since_home'] = ((df['date'] - df['visitor_prev_home']).dt.days).clip(lower=1, upper=15)
df['visitor_days_since_away'] = ((df['date'] - df['visitor_prev_away']).dt.days).clip(lower=1, upper=15)

In [38]:
df

,home_team,visitor_team,favored_open,favored_close,spread_open,spread_close,total_open,total_close,home_ml,visitor_ml,...,close_total_win,date,home_prev_home,home_prev_away,visitor_prev_home,visitor_prev_away,home_days_since_home,home_days_since_away,visitor_days_since_home,visitor_days_since_away
131101_701,Orlando,NewOrleans,V,V,4.0,4.0,194.5,193.5,155,-175,...,over,2013-11-01,2013-10-31,2013-10-31,2013-10-31,2013-10-31,1,1,1,1
131101_703,Charlotte,Cleveland,V,V,1.5,3.5,190.5,188.0,145,-165,...,under,2013-11-01,2013-10-31,2013-10-31,2013-10-31,2013-10-31,1,1,1,1
131101_705,Washington,Philadelphia,H,H,8.5,9.5,196.5,201.0,-530,405,...,over,2013-11-01,2013-10-31,2013-10-31,2013-10-31,2013-10-31,1,1,1,1
131101_707,Atlanta,Toronto,H,H,3.5,3.5,194.0,195.0,-160,140,...,over,2013-11-01,2013-10-31,2013-10-31,2013-10-31,2013-10-31,1,1,1,1
131101_709,Boston,Milwaukee,H,H,2.5,2.5,191.0,185.5,-140,120,...,over,2013-11-01,2013-10-31,2013-10-31,2013-10-31,2013-10-31,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19310_545,GoldenState,LAClippers,V,V,12.5,12.5,226.5,223.5,575,-850,...,over,2020-03-10,2020-03-07,2020-03-03,2020-03-08,2020-03-05,3,7,2,5
19311_547,Atlanta,NewYork,H,H,4.0,5.5,230.0,232.5,-250,200,...,over,2020-03-11,2020-03-09,2020-03-07,2020-03-08,2020-03-10,2,4,3,1
19311_549,Philadelphia,Detroit,H,H,11.0,12.0,215.0,209.5,-800,550,...,over,2020-03-11,2020-02-27,2020-03-07,2020-03-07,2020-03-08,13,4,4,3
19311_551,Miami,Charlotte,H,H,10.0,11.0,211.5,206.5,-700,500,...,over,2020-03-11,2020-03-04,2020-03-08,2020-03-07,2020-03-09,7,3,4,2


In [39]:
df.to_csv('../data/transformed/final_dataframe.csv')